# Overview

**"Incremental Learning"** is a methodology in machine learning where a pre-trained model can adapt and improve its performance over time by learning from new test data and user feedback. This process occurs in real-time production environments without the need to retrain the model from scratch.

________

#### Segment-1: Training a Baseline ML Model for Production 

To begin, import and train the machine learning model from a retrain-compatible library such as `creme`. This model is version 1.0 that gets deployed onto the production environment. 

#### Segment-2: Incrementally Retrain the Production ML Model

Next, load the trained machine learning model and implement incremental retraining by sequentially providing new test data samples. The model retrains on new samples, one at a time, to gain new knowledge on top of already learned patterns. 

#### Segment-3: Apply the Incrementally Retrained ML Model in Production 

Finally, utilize the incrementally retrained model i.e., version 1.1 in the production environment to benefit from its improved knowledge. Repeat the Segments 2 and 3 periodically to keep the model up to date on latest data. 


### DISCLAIMER: This notebook exclusively concentrates on "Iterative Retraining," therefore, the data pre-processing and model fine-tuning steps are kept concise and not extensively elaborated upon.

# Segment-1: Training a Baseline ML Model for Production 

In [1]:
# import libraries 
import pandas as pd
from sklearn.model_selection import train_test_split

import math
import creme
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
from creme import metrics

In [1]:
# import training data 
messages = pd.read_csv('SMSSpamCollection', sep='\t', names=["label", "message"])

In [7]:
messages.shape

(5572, 2)

In [8]:
# split the datast into train and test sets 
message_train,message_test=train_test_split(messages)

In [9]:
message_train

label                                            message
1092   ham  Please da call me any mistake from my side sor...
3001   ham    Id onluy matters when getting on from offcampus
2153   ham  Hi kindly give us back our documents which we ...
1020   ham                    Don know..wait i will check it.
4191   ham                             Sorry, I'll call later
...    ...                                                ...
2245   ham                            No management puzzeles.
3682   ham           LOL what happens in Vegas stays in vegas
2327   ham  The Xmas story is peace.. The Xmas msg is love...
4646   ham                  Are you planning to come chennai?
2349   ham              That seems unnecessarily affectionate

[4179 rows x 2 columns]

In [10]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test=message_test.to_records(index=False)

In [11]:
messages_train

rec.array([('ham', 'Please da call me any mistake from my side sorry da. Pls da goto doctor.'),
           ('ham', 'Id onluy matters when getting on from offcampus'),
           ('ham', 'Hi kindly give us back our documents which we submitted for loan from STAPATI'),
           ...,
           ('ham', 'The Xmas story is peace.. The Xmas msg is love.. The Xmas miracle is jesus.. Hav a blessed month ahead &amp; wish U Merry Xmas...'),
           ('ham', 'Are you planning to come chennai?'),
           ('ham', 'That seems unnecessarily affectionate')],
          dtype=[('label', 'O'), ('message', 'O')])

In [ ]:
# Create Pipeline: Embedding Model and Prediction Model 
model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

In [ ]:
metric=metrics.Accuracy()

# model training 
for label,sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)
   

In [ ]:
# Model accuracy on train data 
metric

In [ ]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messages_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

In [ ]:
# Model accuracy on test data 
test_metric

In [ ]:
# Real-time Prediction using the ML model V 1.0

user_input = "You are receiving this newsletter due to your subscription request."
prediction = model.predict_one(user_input)

print(prediction) # ham or spam 

# Segment-2: Incrementally Retrain the Production ML Model

In [ ]:
new_test_samples = {1: ["Congratulations! You've won a luxury vacation worth $10,000! Click here to claim your prize now!", "spam"], 
                    2: ["Get rich quick with our exclusive investment opportunity! Start earning thousands of dollars in just days!", "spam"], 
                    3: ["Limited time offer: Buy one, get one free on all our miracle weight loss pills! Shed those pounds effortlessly!", "spam"], 
                    4: ["Act now to secure your inheritance from a distant relative! Just provide your bank details to receive your windfall!", "spam"], 
                    5: ["Don't miss out on our amazing deals! Shop now for discounts up to 90% off on brand-name electronics and fashion items!", "spam"], 

                    6: ["Thank you for your recent purchase! Your order has been successfully processed and will be shipped within the next 24 hours.", "ham"], 
                    7: ["Reminder: Your subscription to our newsletter is expiring soon. Click here to renew and stay updated with our latest offers and updates.", "ham"], 
                    8: ["We appreciate your feedback! Please take a moment to complete our survey and help us improve our products and services.", "ham"], 
                    9: ["Invitation: Join us for our upcoming webinar on 'Effective Time Management Strategies'. Register now to reserve your spot!", "ham"], 
                    10: ["Happy Birthday! As a token of our appreciation, enjoy a special discount of 20% off your next purchase. Use code BDAY20 at checkout.", "ham"], 
                   }

In [ ]:
# Incrementally retrain the loaded model on new test samples one-by-one 
for key, value in new_test_samples.items():
    model.fit_one(value[0], value[1])

In [ ]:
# New Output Classes can also be added to the ML model with this apprach 

# Adding new output class - uncertain
uncertain_samples = {1: ["Quick question: Are you interested in boosting your income?", "uncertain"], 
                     1: ["Special offer: Limited-time discount on select products.", "uncertain"], 
                     1: ["Important notice: Action required to update your account information.", "uncertain"], 
                     1: ["Exciting opportunity: Join our exclusive membership program.", "uncertain"], 
                     1: ["Urgent request: Please review and respond to this message promptly.", "uncertain"], 
                   }

for key, value in new_test_samples.items():
    model.fit_one(value[0], value[1])

# Segment-3: Apply the Incrementally Retrained ML Model in Production 

In [ ]:
# Real-time Prediction using the ML model V 1.1

user_input = "Thank you for joining our community! You are receiving this newsletter as a result of your subscription request."
prediction = model.predict_one(user_input)

print(prediction) # ham or spam or uncertain 

__Repeat Segments 2 and 3 for continuous model updates on the latest user data.__